In [ ]:
#Libraries required 

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import datetime

from bs4 import BeautifulSoup
import requests

import json

import pandas as pd
import numpy as np

import pymysql

In [ ]:
# Generate categories and subcategories list of dictionaries

#File with all categories and subcategories id's used to generate the URLS for each categorie and subcategorie, available 
#in the repository
file_path = 'job_listings_container_html.txt'  

# Open the file in read mode
with open(file_path, 'r', encoding='utf-8') as file:
    html_content = file.read()

#Reading the HTML code
subcategories_html = BeautifulSoup(html_content, 'lxml')

subcategories = subcategories_html.find_all('div', class_='up-group-container', attrs={'data-test': 'up-c-group', 'data-test-key': True})

categories_list = []

#Generating the list of lists of dictionaries for the categories and its corresponding subcategories. All with their id
for a, subcategorie in enumerate(subcategories):
    subcategorie_title = subcategorie.find('div', class_='up-group-header').text.strip()
    
    sub2_bucket = subcategorie.find_all('li', attrs={'role': 'option', 'aria-selected': 'false', 'tabindex': '0', 'class': 'up-multi-select up-menu-item'})
    
    subcategories_data = []

    for b, sub2 in enumerate(sub2_bucket):
        b += 1
        ids = sub2.select_one('input[type="checkbox"][tabindex="-1"][id^=""]')
        id_ = ids.get('id')
        
        subcategory_data = {'name': sub2.text.strip(), 'id': id_}
        subcategories_data.append(subcategory_data)
    
    main_category_data = {'main_category': subcategorie_title, 'subcategories': subcategories_data}
    categories_list.append(main_category_data)

#Cleaning the main categories id's
for a, dictionarie in enumerate(categories_list):
    categories_list[a]['subcategories'][0]['id'] = categories_list[a]['subcategories'][0]['id'][4:]


In [ ]:
#Verifying all categories and subcategories dictionarie
categories_list

In [ ]:
#Defining the function to iterate the url of each subcategorie for all existing web pages
def jobs_scraper(cat, sub, cat_sub_url):
    total_jobs = []
    total_skils = []
    i = 0
    
    #Run while there are available web pages
    while True:
        i += 1
        cat_sub_url = cat_sub_url + f'&page={i}'
        driver = webdriver.Chrome()
        
        #Pull all job details for each web page: JobID, Category, Subcategory, job title, Job Type, Payment, Duration, Hours/Week, 
        #Experience, Job description, Posted date, scraped date
        try:
            #Based on the web page URl wait fot the dynamicalli loaded job posts to appear
            driver.get(cat_sub_url)
            wait = WebDriverWait(driver, 10)
            job_listings_container = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-test="job-tile-list"]')))
            job_listings_container_html = job_listings_container.get_attribute("outerHTML")
            parser = BeautifulSoup(job_listings_container_html, 'lxml')
            sample_job = parser.find('section', class_="up-card-section up-card-list-section up-card-hover")
            All_jobs = parser.find_all('section', class_="up-card-section up-card-list-section up-card-hover")
            
            jobs_p = []
            jobs_sk = []
            
            #Iterate through all job posts listed in each web page to extract all details for each job
            for jobs in All_jobs:
                Id2 = int(jobs['data-test-key'])
                Job_type_details = jobs.find('strong', {"data-test": "job-type"})
                Experience = jobs.find('span', {"data-test": "contractor-tier"}).text
                Posted = jobs.find('span', {"data-test": "UpCRelativeTime"}).text
                scraped = datetime.datetime.now().strftime("%Y/%m/%d %H:%M:%S")
                Description = jobs.find('span', {"data-test": "job-description-text"}).text.strip()
                
                #Determine whether it's a hourly or project job
                if Job_type_details.text == 'Fixed-price':
                    Job_type = 'Project'
                    Budget = jobs.find('span', {"data-test": "budget"}).text.strip()
                    Duration = 'Fixed-price'
                    Hours_Week = 'Fixed-price'
                else:
                    Job_type = 'Job'
                    Budget = Job_type_details.text[8:]
                    Duration = jobs.find('span', {"data-test": "duration"}).text.strip().split(', ')[0]
                    Hours_Week = jobs.find('span', {"data-test": "duration"}).text.strip().split(', ')[1]
                
                jobs_att = (
                    Id2, cat, sub, jobs.h3.text, Job_type, Budget, Duration, Hours_Week, Experience, Description, Posted,
                    scraped)
                
                jobs_p.append(jobs_att)
                skills = jobs.find_all('a', class_="up-skill-badge text-muted")
                
                #iterate all skills for each job post
                for skill in skills:
                    skills_pair = (Id2, skill.text)
                    jobs_sk.append(skills_pair)
            
            #Add the jobs being aded for each web page to the main list
            total_jobs.extend(jobs_p)
            total_skils.extend(jobs_sk)
        
        #When there's no more web pages available stop the iteration
        except TimeoutException:
            driver.save_screenshot('C:/Users/Acer/Desktop/Jupyter_para_DS/Upwork/creenshot.png')
            print('Iteration completed')
            break
    
    #Return the main lists for all jobs and its skills
    return total_jobs, total_skils


In [ ]:
%%time
#Selecting the Data & Analytics subcategories to generate the URL's for each and iterate them through the previously 
#defined function to scrape eache subcategorie

website = 'https://www.upwork.com/nx/jobs/search/?sort=recency'
cat = categories_list[3]['main_category']
main = categories_list[3]['subcategories'][0]['id']

tj = []
tk = []

for uid in categories_list[3]['subcategories'][1:]:
    sub_id = uid['id']
    sub_name = uid['name']
    sub_website = website + f'&category2_uid={main}&subcategory2_uid={sub_id}'
    
    print('Scraping:', '\n', f'Categorie: {cat}.', '\n', f'Subcategorie: {sub_name}', '\n', 
          f'URL: {sub_website}', '\n')
    
    jobs, skills = jobs_scraper(cat, sub_name, sub_website)
    tj.extend(jobs)
    tk.extend(skills)
    

In [ ]:
# Connecting MySQL with Python to ingest all jobs into the job table
host = '127.0.0.1'
user = 'root'
password = input('Enter your MySQL password: ')
schema = 'scraping'

# Establish a connection to the database
connection = pymysql.connect(host=host, user=user, password=password, db=schema)

try:
    cursor = connection.cursor()

    # SQL query to insert data into the 'jobs' table
    sql = "INSERT INTO jobs VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

    cursor.executemany(sql, tj)

    connection.commit()

finally:
    # Close the cursor and the connection
    cursor.close()
    connection.close()


In [ ]:
# Connecting MySQL with Python to ingest all skills into the skills table
host = '127.0.0.1'
user = 'root'
password = input('Enter your MySQL password: ')
schema = 'scraping'

connection = pymysql.connect(host=host, user=user, password=password, db=schema)

try:
    cursor = connection.cursor()

    # SQL query to insert data into the 'jobs' table
    sql = "INSERT INTO jobs VALUES (%s, %s, %s)"

    cursor.executemany(sql, tk)

    connection.commit()

finally:
    cursor.close()
    connection.close()
